# Distilling Knowledge in Multiple Students Using Generative Models

In [1]:
# %tensorflow_version 1.x
# !pip install --upgrade opencv-python==3.4.2.17
import numpy as np
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K
# import os
from tensorflow.keras.datasets import fashion_mnist,mnist,cifar10
# import keras.backend as K
from tensorflow.keras.layers import Conv2D,Activation,BatchNormalization,UpSampling2D,Embedding,ZeroPadding2D, Input, Flatten, Dense, Reshape, LeakyReLU, Dropout,MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import regularizers

from tensorflow.keras.utils import Progbar
from keras.initializers import RandomNormal
import random
from sklearn.model_selection import train_test_split
# from keras.utils import np_utils
from tensorflow.keras import utils as np_utils



In [2]:
#Loading and splitting the dataset into train, validation and test
nb_classes = 10

(X_Train, y_Train), (X_test, y_test) = fashion_mnist.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_Train, y_Train, test_size=0.20)
# convert y_train and y_test to categorical binary values 
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_val = np_utils.to_categorical(y_val, nb_classes)

X_train = X_train.reshape(48000, 28, 28, 1)
X_val = X_val.reshape(12000, 28, 28, 1)

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')

# Normalize the values
X_train /= 255
X_val /= 255


#Creating a teacher network
input_shape = (28, 28, 1) # Input shape of each image

teacher = Sequential()
teacher.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
teacher.add(Conv2D(64, (3, 3), activation='relu'))
teacher.add(MaxPooling2D(pool_size=(2, 2)))

teacher.add(Dropout(0.25)) # For reguralization

teacher.add(Flatten())
teacher.add(Dense(256, activation='relu'))
teacher.add(Dense(256, activation='relu', name="dense_1"))

teacher.add(Dropout(0.5)) # For reguralization

teacher.add(Dense(nb_classes, name = 'dense_2'))
teacher.add(Activation('softmax')) # Note that we add a normal softmax layer to begin with

teacher.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(teacher.summary())

4423680/4422102 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2359552   
___________________________________________________________

In [3]:
teacher.load_weights("Teacher_FMNIST_92.h5")

In [4]:
# Train the teacher model as usual
epochs = 0
batch_size = 256
teacher.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, Y_val))


In [25]:
#checking teachers test accuracy
X_test = X_test.reshape(10000, 28, 28, 1)
X_test = X_test.astype('float32')
X_test /= 255

Y_test = np_utils.to_categorical(y_test, nb_classes)

teacher.evaluate(X_test,Y_test)

313/313 [==============================] - 1s 4ms/step - loss: 0.3516 - accuracy: 0.9239


[0.351641982793808, 0.9239000082015991]

In [27]:
teacher_WO_Softmax = Model(teacher.input, teacher.get_layer('dense_1').output)
train_dense = teacher_WO_Softmax.predict(X_train)
val_dense = teacher_WO_Softmax.predict(X_val)



# 1 Student case
# ---------------------------------------------
s1Train=train_dense[:,:256]
s1Val=val_dense[:,:256]

In [28]:
def define_model(name):

    student1 = Sequential()
    student1.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28, 28, 1),kernel_initializer='normal', name=name))
    student1.add(Conv2D(32, (3, 3), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Conv2D(16, kernel_size=(3, 3),activation='relu',kernel_initializer='normal'))
    student1.add(Conv2D(16, (3, 3), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Dropout(0.25)) # For reguralization
    student1.add(Flatten())
    student1.add(Dense(64, activation='relu'))
    student1.add(Dropout(0.3))
    student1.add(Dense(256,name='req'+name))

    student1.compile('adam',loss='mse',metrics=['accuracy'])
    return student1



In [29]:
s1=define_model("s1")

In [30]:
batch_size = 128
s1.fit(X_train, s1Train,
          batch_size=batch_size,
          epochs=10,
          verbose=1, validation_data=(X_val, s1Val))

Epoch 1/10
375/375 [==============================] - 2s 5ms/step - loss: 0.9835 - accuracy: 0.1146 - val_loss: 0.3117 - val_accuracy: 0.3374
Epoch 2/10
375/375 [==============================] - 3s 8ms/step - loss: 0.3813 - accuracy: 0.2957 - val_loss: 0.2550 - val_accuracy: 0.3593
Epoch 3/10
375/375 [==============================] - 3s 8ms/step - loss: 0.3230 - accuracy: 0.3345 - val_loss: 0.2275 - val_accuracy: 0.3766
Epoch 4/10
375/375 [==============================] - 4s 11ms/step - loss: 0.2968 - accuracy: 0.3473 - val_loss: 0.2080 - val_accuracy: 0.3882
Epoch 5/10
375/375 [==============================] - 4s 10ms/step - loss: 0.2804 - accuracy: 0.3546 - val_loss: 0.2110 - val_accuracy: 0.3906
Epoch 6/10
375/375 [==============================] - 3s 9ms/step - loss: 0.2736 - accuracy: 0.3620 - val_loss: 0.1904 - val_accuracy: 0.4034
Epoch 7/10
375/375 [==============================] - 4s 10ms/step - loss: 0.2655 - accuracy: 0.3662 - val_loss: 0.1852 - val_accuracy: 0.4095
Epo

### **1 Student**

In [31]:
o1=s1.get_layer("reqs1").output
output=Activation('relu')(o1)
output2=Dropout(0.5)(output)
output3=Dense(10,activation="softmax", name="d1")(output2)
mm1=Model(s1.get_layer('s1').input, output3)
my_weights=teacher.get_layer('dense_2').get_weights()

In [32]:
mm1.get_layer('d1').set_weights(my_weights)

In [33]:
for l in mm1.layers[:len(mm1.layers)-1]:
    l.trainable=False


In [34]:
mm1.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [36]:
batch_size = 128
mm1.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1, validation_data=(X_val, Y_val))

Epoch 1/5
375/375 [==============================] - 2s 4ms/step - loss: 0.4535 - accuracy: 0.8353 - val_loss: 0.3501 - val_accuracy: 0.8712
Epoch 2/5
375/375 [==============================] - 2s 4ms/step - loss: 0.4555 - accuracy: 0.8333 - val_loss: 0.3498 - val_accuracy: 0.8727
Epoch 3/5
375/375 [==============================] - 3s 7ms/step - loss: 0.4544 - accuracy: 0.8347 - val_loss: 0.3511 - val_accuracy: 0.8723
Epoch 4/5
375/375 [==============================] - 2s 6ms/step - loss: 0.4593 - accuracy: 0.8320 - val_loss: 0.3522 - val_accuracy: 0.8724
Epoch 5/5
375/375 [==============================] - 1s 2ms/step - loss: 0.4567 - accuracy: 0.8334 - val_loss: 0.3522 - val_accuracy: 0.8691


In [37]:
loss, acc = mm1.evaluate(X_test, Y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.3730 - accuracy: 0.8629
